In [48]:

import os
import copy
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM
import shap
import scipy as sp
import nlp
import torch
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
pd.set_option("max_rows", None)
from shap.utils import safe_isinstance

ModuleNotFoundError: No module named 'transformers.examples'

In [45]:
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")

model =  AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6").cuda()

In [8]:
dataset = nlp.load_dataset('xsum',split='train')

WARNING - Using custom data configuration default


Dataset xsum downloaded and prepared to C:\Users\ryserrao\.cache\huggingface\datasets\xsum\default\1.1.0\0140182f1073b990fc0b6e7159a845ef3eaeff2fe8c155bd96cc31236ddf5fb3. Subsequent calls will reuse this data.


In [20]:
ARTICLE_TO_SUMMARIZE=dataset['document'][0].split(' ')

In [12]:
dataset['summary'][0]

'Sony has told owners of older models of its PlayStation 3 console to stop using the machine because of a problem with the PlayStation Network.\n'

In [19]:
len(tokenizer.encode('The problem is greatly affecting people using the older versions of the PlayStation 3, called the "Fat" model.The problem isn\'t affecting the newer PS3 Slim systems that have been on sale since September last year.Sony have also said they are aiming to have the problem fixed shortly but is advising some users to avoid using their console for the time being."We hope to resolve this problem within the next 24 hours," a statement reads. "In the meantime, if you have a model other than the new slim PS3, we advise that you do not use your PS3 system, as doing so may result in errors in some functionality, such as recording obtained trophies, and not being able to restore certain data."We believe we have identified that this problem is being caused by a bug in the clock functionality incorporated in the system."The PlayStation Network is used by millions of people around the world.It allows users to play their friends at games like Fifa over the internet and also do things like download software or visit online stores.\n'))

208

In [63]:
inputs = torch.tensor(tokenizer.encode(ARTICLE_TO_SUMMARIZE, max_length=1024,truncation=True,padding="longest")).cuda()

In [64]:
inputs

tensor([    0,   133, 41045,   354,     3, 11970, 10928,   627, 33780, 44784,
         1116,   627,     3,     3,  4155,   627,     3,     3, 41045,     3,
            3,   627,     3,     3,     3,     3,  6025, 11990, 35507,   261,
        25248, 30117, 16913, 13751,     3, 11990, 19726, 28613, 10010,  1322,
            3,   560, 11990,   627, 41045, 43713,     3,  4297,   354,     3,
        12465, 31964,   560, 40623, 10928, 25017, 48028,  1990,   627,   958,
            3,     3,   560,     3,  9226, 41045, 34451,   627, 25616,  1978,
            3,   102, 29963,     3,     3,   627,     3,  1594,  6968, 11990,
          102, 21818,  7443,  5652,   627,  4651,     3,     3,  1694,     3,
         6025,  6968,  5016,  3654,  3698, 16625,     3,     3,   281, 30012,
         2527, 12488, 43155,   179, 48487,   179, 12465,     3, 16918,   281,
            3,     3,     3,   463,  3654,  9442,   868,   560,     3, 31097,
            3,     3,  1694, 11990, 23788,  6025,  9226, 41045, 

In [65]:
tokenizer.mask_token

'<mask>'

In [67]:
summary_ids = model.generate(inputs,num_beams=4, max_length=10, early_stopping=True)

AssertionError: Input prompt should be of shape (batch_size, sequence length).

In [49]:
model.config.task_specific_params

{'summarization': {'early_stopping': True,
  'length_penalty': 2.0,
  'max_length': 142,
  'min_length': 56,
  'no_repeat_ngram_size': 3,
  'num_beams': 4}}

In [51]:
prefix=None
prefix = prefix or getattr(model.config, "prefix", "") or ""

In [52]:
prefix

' '

In [43]:
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

[' CNN.com will feature iRep', ' Problem is a big problem in the', ' CNN.com will feature iRep', 'affecting.affects', ' People are famous for their work in', 'using .using Facebook to create a', ' The U.S. House of', ' Older than 50 years older, older', "versions of the world's most popular", " Of course. of the world's", ' The U.S. House of', ' PlayStation is a PlayStation 3 game', ' 3.3,3.3', 'called.called .called . Called', ' The U.S. House of', ' "Fat" is the name of', " The model of the world's most", ' Problem is a big problem in the', ' CNN.com will feature iRep', 'affecting.affects', ' The U.S. House of', 'newer videos have been released in', ' PS3 will be available in the', " Slimlim is the world's", 'systems.systems .system', 'that.that .that is a', ' Have you ever had a problem?', 'been.been .been . Been', 'on.on . CNN.com', 'sale.sale . Sale . Sale', ' Since then, there has been a', ' September 1: September 2: September', " Last week's events in Washington were", ' Sony an

In [41]:
dataset['document'][0]

'The problem is affecting people using the older versions of the PlayStation 3, called the "Fat" model.The problem isn\'t affecting the newer PS3 Slim systems that have been on sale since September last year.Sony have also said they are aiming to have the problem fixed shortly but is advising some users to avoid using their console for the time being."We hope to resolve this problem within the next 24 hours," a statement reads. "In the meantime, if you have a model other than the new slim PS3, we advise that you do not use your PS3 system, as doing so may result in errors in some functionality, such as recording obtained trophies, and not being able to restore certain data."We believe we have identified that this problem is being caused by a bug in the clock functionality incorporated in the system."The PlayStation Network is used by millions of people around the world.It allows users to play their friends at games like Fifa over the internet and also do things like download software o